In [ ]:
from scipy.io import loadmat
import pandas as pd
import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
import wfdb
import re
from os.path import exists
import matplotlib.pyplot as plt

In [ ]:
dataset = []
y = []
for n in range(1, 10):
    for j in range(0, 10):
        for i in range(1, 1000):
            if(exists('physionet/WFDBRecords/' + f"{n:02}" +  '/' + f"{n:02}" + str(j) +  '/JS' + f"{ 1000*(n-1) + 100*(j) + i :05}"  + '.hea')):
                try:
                    ecg_record = wfdb.rdsamp('physionet/WFDBRecords/' + f"{n:02}" +  '/' + f"{n:02}" + str(j) +  '/JS' + f"{ 1000*(n-1) + 100*(j) + i :05}")
                except ValueError:
                    continue
                ecg = torch.from_numpy(ecg_record[0])
                ecg_record[1]['comments'][2]
                numbers = re.findall(r'\d+', ecg_record[1]['comments'][2])
                output_array = list(map(int, numbers))
                dataset.append(ecg_record[0])
                y.append(output_array)

In [ ]:
dataset = np.array(dataset)
dataset = dataset.astype( np.double, copy=False)

In [ ]:
b = np.zeros([len(y),len(max(y,key = lambda x: len(x)))])
for i,j in enumerate(y):
    b[i][0:len(j)] = j
b = np.array(b, dtype= np.double)

In [ ]:
classes = {
        426177001: 1,
        426783006: 2,
        164889003: 3,
        427084000: 4,
        164890007: 5,
        427393009: 6,
        426761007: 7,
        713422000: 8,
        233896004: 9,
        233897008: 0
}

In [ ]:
max(y ,key = lambda x: len(x))
lengths = y

In [ ]:
dataset = dataset.astype(np.double)
X = torch.from_numpy(dataset[:, :, :])
print(X.shape)
X = X.permute(0, 2, 1)
X.shape

In [ ]:
labels = list(set(i for j in b for i in j))
dict = {}
for i, j in enumerate(labels):
    dict[j] = i

for i in range(len(y)):
    for j in y[i]:
        if int(j) in classes:
            # classes[int(j)] += 1
            print('a')
            y[i] = j
            break

In [ ]:
for i in range(len(y)):
    y[i] = classes[y[i]]

In [ ]:
# def to_onehot(labels, n_categories, dtype=torch.double):
#     batch_size = len(labels)
#     one_hot_labels = torch.zeros(size=(batch_size, n_categories), dtype=dtype)
#     for i, label in enumerate(labels):
#         # Subtract 1 from each LongTensor because your
#         # indexing starts at 1 and tensor indexing starts at 0
#         label = torch.LongTensor(label) - 1
#         one_hot_labels[i] = one_hot_labels[i].scatter_(dim=0, index=label, value=1.)
#     return one_hot_labels

# Y = torch.nn.functional.one_hot(torch.from_numpy(np.array(y)), num_classes= 10) 

In [ ]:
train_size = 7500
test_size = 1400
dataset = dataset.astype(np.double)
Y = torch.from_numpy(np.array(y))
X_train = (X[:train_size, :, :])
Y_train = (Y[:train_size])
X_test = (X[train_size:test_size + train_size, :, :])
Y_test = (Y[train_size:test_size + train_size])

In [ ]:
train_batch_size = 200
test_batch_size = 100
from torch.utils.data import DataLoader
train_dataset = torch.utils.data.TensorDataset(X_train, Y_train)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True)
test_dataset = torch.utils.data.TensorDataset(X_test, Y_test)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=test_batch_size, shuffle=True)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class myModule(nn.Module):

    def __init__(self, c_in, subsample=False, c_out=-1):
        super().__init__()

        self.layer1 = nn.Sequential(
            nn.Conv1d(c_in, 16, kernel_size=5, padding=1, stride=1, bias=False),  # No bias needed as the Batch Norm handles it
            nn.BatchNorm1d(16),
            nn.ReLU(),
            nn.MaxPool1d(2),
        )
        self.layer2 = nn.Sequential(
            nn.Conv1d(16, 16, kernel_size=5, padding=1, stride=1, bias=False),  # No bias needed as the Batch Norm handles it
            nn.BatchNorm1d(16),
            nn.ReLU(),
            nn.MaxPool1d(2),
        )
        self.layer3 = nn.Sequential(
            nn.Conv1d(16, 32, kernel_size=5, padding=1, stride=1, bias=False),  # No bias needed as the Batch Norm handles it
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.MaxPool1d(4),
        )
        self.layer4 = nn.Sequential(
            nn.Conv1d(32, 32, kernel_size=3, padding=1, stride=1, bias=False),  # No bias needed as the Batch Norm handles it
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.MaxPool1d(2),
        )
        self.layer5 = nn.Sequential(
            nn.Conv1d(32, 64, kernel_size=3, padding=1, stride=1, bias=False),  # No bias needed as the Batch Norm handles it
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.MaxPool1d(2),
        )
        self.layer6 = nn.Sequential(
            nn.Conv1d(64, 64, kernel_size=3, padding=1, stride=1, bias=False),  # No bias needed as the Batch Norm handles it
            nn.BatchNorm1d(64),
            nn.ReLU(),
            # nn.MaxPool1d(4)
        )
        self.spatial_layer = nn.Sequential(
            nn.Conv1d(64, 12, kernel_size=3, padding=1, stride=0, bias=False),  # No bias needed as the Batch Norm handles it
            nn.BatchNorm1d(c_out),
            nn.ReLU(),
        )
        self.fc1 = nn.Sequential(
            nn.Linear(in_features=1216, out_features=608),  # No bias needed as the Batch Norm handles it
            nn.BatchNorm1d(608),
            nn.ReLU(),
            nn.Dropout(p=0.2)
        )
        self.fc2 = nn.Sequential(
            nn.Linear(in_features=608, out_features=304),  # No bias needed as the Batch Norm handles it
            nn.BatchNorm1d(304),
            nn.ReLU(),
            nn.Dropout(p=0.2)
        )
        self.fc3 = nn.Sequential(
            nn.Linear(in_features=304, out_features=10),  # No bias needed as the Batch Norm handles it
            nn.BatchNorm1d(10),
            nn.ReLU(),
            nn.Dropout(p=0.2)
        )
        self.flat = nn.Flatten()
        
        self.gradients = None,

        self.maxxy  = nn.MaxPool1d(4)

        # hook for the gradients of the activations
    def activations_hook(self, grad):
        self.gradients = grad

    # def forward(self, x):
    def forward(self, x):
        x = self.layer1(x)
        # print(x.shape)

        x = self.layer2(x)
        # print(x.shape)

        x = self.layer3(x)
        # print(x.shape)

        x = self.layer4(x)
        # print(x.shape)

        x = self.layer5(x)
        # print(x.shape)

        x = self.layer6(x)

        print(x.shape)
        x = self.maxxy(x)

        x = self.flat(x)

        x = self.fc1(x)

        x = self.fc2(x)

        x = self.fc3(x)

        x = nn.Softmax(dim=1)(x)

        # x = x.argmax(1)
        
        return x
    
    def get_activations_gradient(self):
        return self.gradients
    
    def get_activations(self, x):
        return self.features_conv(x)


my_nn = myModule(12 , c_out = 1)
my_nn = my_nn.double()

In [ ]:
for name, param in my_nn.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

In [ ]:
import torch.optim as optim
learning_rate = 0.001
num_epochs = 50
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(my_nn.parameters(), lr=learning_rate)

In [ ]:
del X
del Y

In [ ]:
# Training
for epoch in range(num_epochs):
    my_nn.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = my_nn(inputs)
        loss = loss_fn(outputs.float(), labels.long())
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
    avg_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {avg_loss:.4f}")

In [ ]:
outputs.shape

In [ ]:
my_nn.eval()
pred_probab = my_nn(X_train[201:400])
output = pred_probab.argmax(1)
correct = (output == Y_train[201:400])

In [ ]:
accurate = 0
total = 0
for i in correct:
  if(i):
    accurate += 1
  total += 1
acc = (accurate/total)*100
print("Accuracy = " + str(acc))

In [ ]:
# Test Accuracy 

In [ ]:
pred_probab = my_nn(X_test[150:300])
output = pred_probab.argmax(1)
correct = (output == Y_test[150:300])

In [ ]:
accurate = 0
total = 0
for i in correct:
  if(i):
    accurate += 1
  total += 1
acc = (accurate/total)*100
print("Accuracy = " + str(acc) )

In [ ]:
#Saving the model parameters. Parameters are saved, hence Don't run 

PATH1 = 'physionet/my_nn.txt'
PATH2 = 'physionet/my_nn2.txt'
torch.save(my_nn.state_dict(), PATH1)
torch.save(my_nn, PATH2)

In [ ]:
PATH1 = 'physionet/my_nn.txt'
model = myModule(12 , c_out = 1)
model.load_state_dict(torch.load(PATH1))
model.eval()
model = model.double()

In [ ]:
Y_train == 3

In [ ]:
index = [i for i ,e in enumerate(Y_train) if e == 3]

In [ ]:
index[:100]

In [ ]:
from torchcam.methods import GradCAMpp, SmoothGradCAMpp
from scipy.interpolate import CubicSpline
import seaborn as sns

# i = 9
# cams = GradCAMpp(model, ["layer6"])
# out = model(X_train[i:i + 1])
# cams

In [ ]:
# cam = cams(class_idx= 1,scores = out)
# for t in cam:
#   print(t.shape)

In [ ]:
# out.argmax(dim = 1)[0]

In [ ]:
# from scipy.interpolate import CubicSpline
# plt.figure(figsize=(20, 10))
# large_array = (X_train[i][0].tolist())
# small_array = np.array(cam[0].tolist()).reshape([77])

# x_large = np.arange(len(large_array))
# x_small = np.linspace(0, len(large_array), len(small_array))

# plt.plot(x_large, large_array, label='Large Array')

# cubic_interpolator = CubicSpline(x_small, small_array)
# interpolated_data = cubic_interpolator(x_large)

# # Overlay the heatmap of the smaller array
# plt.scatter(x_small, small_array, c=small_array, cmap='hot', s=100, alpha=0.7, label='Heatmap')
# # for i, val in enumerate(small_array):
# #     plt.vlines(x_small[i], ymin=np.min(large_array), ymax=val, colors='red', linewidth=2, alpha=0.5)

# # Add colorbar for the heatmap
# cbar = plt.colorbar()
# cbar.set_label('Heatmap Intensity')

# # Add labels and legend
# plt.xlabel('SmoothGradCAMpp')
# plt.ylabel('Value')
# plt.legend()

In [ ]:
for i in index:
    cams = GradCAMpp(model, ["layer6"])
    out = model(X_train[i:i + 1])
    cam = cams(class_idx= 1,scores = out)

    large_array = (X_train[i][0].tolist())
    small_array = np.array(cam[0].tolist()).reshape([77])

    x_large = np.arange(len(large_array))
    x_small = np.linspace(0, len(large_array), len(small_array))

    # plt.plot(x_large, large_array, label='Large Array')

    cubic_interpolator = CubicSpline(x_small, small_array)
    interpolated_data = cubic_interpolator(x_large)

    sns.set(rc={'figure.figsize':(40,30)})

    time_series_data = large_array

    heatmap_data = interpolated_data
    fig, ax1 = plt.subplots()

    # Plot the time series data

    ax1.plot(time_series_data, color='blue', label='Time Series Data')
    ax1.set_ylabel('Time Series', color='blue')
    ax1.tick_params(axis='y', labelcolor='blue')

    # Create a second y-axis for the heatmap
    ax2 = ax1.twinx()
    sns.heatmap([heatmap_data], ax=ax2, cmap='coolwarm', cbar=False, alpha = 0.1)

    # Set labels and title
    ax2.tick_params(axis='y', labelcolor='red')

    # Display the plot
    plt.savefig('foo' + str(i) + '.png')
    plt.show()


In [ ]:
pred_probab = model(X_test[400:600])
output = pred_probab.argmax(1)
correct = (output == Y_test[400:600])

In [ ]:
accurate = 0
total = 0
for i in correct:
  if(i):
    accurate += 1
  total += 1
acc = (accurate/total)*100
print("Accuracy = " + str(acc) )